In [ ]:
using System.IO;
using System.Globalization;
using System.Collections;
using System.Linq;

//https://stackoverflow.com/questions/4269737/function-convert-hex-string-to-bitarray-c-sharp
static BitArray ConvertHexToBitArray(string hexData)
{
    if (hexData == null)
        return null; // or do something else, throw, ...

    BitArray ba = new BitArray(4 * hexData.Length);
    for (int i = 0; i < hexData.Length; i++)
    {
        byte b = byte.Parse(hexData[i].ToString(), NumberStyles.HexNumber);
        for (int j = 0; j < 4; j++)
        {
            ba.Set(i * 4 + j, (b & (1 << (3 - j))) != 0);
        }
    }
    return ba;
}
//https://stackoverflow.com/questions/5283180/how-can-i-convert-bitarray-to-single-int
static int getIntFromBitArray(BitArray bitArray)
{
    int value = 0;
    int last = bitArray.Count-1;
    for (int i = 0; i < bitArray.Count; i++)
    {
        if (bitArray[last-i])
            value += Convert.ToInt16(Math.Pow(2, i));
    }

    return value;
}

In [ ]:
record Packet
{
    public int version;
    public int type;
    public long? literalvalue=null;
    public List<Packet> operands = null;
    public int? numberp = null;
    public int? lengthp = null;
    public int inipos;
    public int endpos;

    public Packet (BitArray input, int pos =0)
    {
        inipos = pos;
        BitArray temp = new BitArray(3);
        temp[0]=input[pos+0];temp[1]=input[pos+1];temp[2]=input[pos+2];
        version = getIntFromBitArray(temp);
        temp[0]=input[pos+3];temp[1]=input[pos+4];temp[2]=input[pos+5];
        type = getIntFromBitArray(temp);
        //Console.WriteLine($"packet {pos} V{version} T{type}");
        if (type!=4) //operation
        {
            operands=new();
            if (input[pos+6]) //11 number of subpackets
            {
                var numberpa = new BitArray(11);
                for (var i=0; i<11; i++) numberpa[i]=input[pos+7+i];
                numberp = getIntFromBitArray(numberpa);
                int np = pos+7+10;
                for (var i =0; i<numberp; i++) 
                {
                    operands.Add (new Packet(input, np+1));
                    np=operands.Last().endpos;
                }
                endpos = np;
            }
            else //15 total length in bits
            {
                var lengthpa = new BitArray(15);
                for (var i=0; i<15; i++) lengthpa[i]=input[pos+7+i];
                lengthp = getIntFromBitArray(lengthpa);
                int np = pos+7+14;
                int inip = np;
                while (np-inip<lengthp) 
                {
                    operands.Add (new Packet(input, np+1));
                    np=operands.Last().endpos;
                }
                endpos = np;
            }
        }
        else //literal value
        {
            int np = pos+5+1;
            literalvalue = ToNumber(input, ref np);
            endpos = np;
        }
    }
    private long ToNumber(BitArray input, ref int pos)
    {
        int inip = pos;
        var c = 0;
        StringBuilder sb = new();

        void CopyFour() {for (var i=0;i<4;i++) sb.Append(input[inip+c+i+1]?"1":"0"); c+=5;}

        while (input[pos+c])
            CopyFour();
        CopyFour();
        pos+=c-1;
        return Convert.ToInt64(sb.ToString(),2);
    }
    public void printinfo(bool nested=false, int tab=0)
    {
        var t = new string(' ',tab);
        Console.Write($"{t}V{version} T{type} f{inipos} t{endpos} ");
        if (literalvalue is not null)
            Console.WriteLine($"L{literalvalue}");
        else if (numberp is not null)
            Console.WriteLine($"#{numberp}");
        else
            Console.WriteLine($"l{lengthp}");
        if (nested && (operands?.Any()??false))
            foreach (var op in operands)
                op.printinfo(true, tab+2);
    }
    public int addversion()
    {
        var v = version;
        if ((operands?.Any()??false))
            foreach (var op in operands)
                v+=op.addversion();
        return v;
    }
    public long calculate() => type switch
        {
            0 => operands?.Sum(x=>x.calculate())??0,
            1 => operands?.Aggregate(1L,(p,n)=>p*n.calculate())??0,
            2 => operands?.Min(x=>x.calculate())??0,
            3 => operands?.Max(x=>x.calculate())??0,
            4 => literalvalue??0,
            5 => operands.First().calculate()>operands.Last().calculate()?1:0,
            6 => operands.First().calculate()<operands.Last().calculate()?1:0,
            7 => operands.First().calculate()==operands.Last().calculate()?1:0,
            _ => 0
        };
}

In [ ]:
var filedata = File.ReadLines(@"..\day16.input").First();

var ba = ConvertHexToBitArray(filedata);
/*foreach (bool b in ba)
    Console.Write(b?"1":"0");
Console.WriteLine();*/

var pa = new Packet(ba);
//pa.printinfo(true);
Console.WriteLine(pa.addversion());
Console.WriteLine(pa.calculate());
//110100101111111000101000 -> 24
//0011100000000000011011 11010001010 01010010001001000000000


963
1549026292886


In [ ]:
int pos = 0;
foreach (bool b in ba)
    Console.Write(b?"1":"0");
Console.WriteLine();
BitArray temp = new BitArray(3);
temp[0]=ba[pos+0];temp[1]=ba[pos+1];temp[2]=ba[pos+2];
foreach (bool b in temp)
    Console.Write(b?"1":"0");
Console.WriteLine();
Console.WriteLine(getIntFromBitArray(temp));
int intba(BitArray bitArray)
{
    int value = 0;

    for (int i = 0; i < bitArray.Count; i++)
    {
        if (bitArray[i])
            value += Convert.ToInt16(Math.Pow(2, i));
    }

    return value;
}
Console.WriteLine(intba(temp));
public static ulong BitArrayToU64(BitArray ba)
{
    var len = Math.Min(64, ba.Count);
    ulong n = 0;
    for (int i = 0; i < len; i++) {
        if (ba.Get(i))
            n |= 1UL << i;
    }
    return n;
}
Console.WriteLine(BitArrayToU64(temp));
//Console.WriteLine($"{pos} {ba[pos+3]}ba[pos+4];temp[2]=ba[pos+5]")
temp[0]=ba[pos+3];temp[1]=ba[pos+4];temp[2]=ba[pos+5];
foreach (bool b in temp)
    Console.Write(b?"1":"0");
Console.WriteLine();
Console.WriteLine(getIntFromBitArray(temp));


110100101111111000101000
110
6
3
3
100
4
